# Import useful libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn import metrics
import matplotlib.pyplot as plt
from functools import reduce
import math
from collections import Counter
import csv
from datetime import datetime
import re
import copy
import json
from datetime import timedelta
from operator import itemgetter
%matplotlib inline

Creation of check point (a print plus the time)

In [ ]:
def now():
    return str(datetime.now().time())[:8]
def pr(strToPrint):
    print(now() + ' '+ strToPrint)

    
# from IPython.display import Audio
# sound_file = 'beep.wav'

## Importing data

Importing a sample of the dataset:

In [ ]:
pickle_filename = os.path.join('data','head_100k_pickle.pkl')
tw = pd.read_pickle(pickle_filename)

Importing the whole dataset

In [ ]:
columns_header = ['id', 'userId', 'createdAt', 'text', 'longitude', 'latitude', 'placeId',
                  'inReplyTo', 'source', 'truncated', 'placeLatitude', 'placeLongitude', 'sourceName', 'sourceUrl',
                 'userName', 'screenName', 'followersCount', 'friendsCount', 'statusesCount',
                 'userLocation']

filename = os.path.join('data','twex.tsv') # 'sample.tsv')
pr('Starting to read file... (3 min)')
tw = pd.read_csv(filename, sep='\t', encoding='utf-8', escapechar='\\', names=columns_header,
                      quoting=csv.QUOTE_NONE, na_values='N', header=None)

pr('File is loaded!')
# Audio(url=sound_file, autoplay=True)

In [ ]:
tw.head(2)

## Extrating hashtags

In [ ]:
def extract_hashtags(text):
    ht_list = re.findall(r"#(\w+)", text)
    non_empty_hts = list(filter((lambda ht: ht != []), ht_list))
    lowerCharList = [ht.lower() for ht in non_empty_hts]
    return lowerCharList

In [ ]:
pr('Making hashtags... (2 min)')
tw['hashtag'] = np.nan
tw.hashtag = tw.text.apply(lambda x: extract_hashtags(str(x)))
twh = tw.ix[tw.hashtag.apply(lambda x: len(x) != 0)]
pr('Done.')

In [ ]:
twh.head(2)

## Cleaning data and making date index

We drop tweet which not contains values for text or createdAt as this is mandatory information to privide to our model.

In [ ]:
tw1 = twh.dropna(axis=0, how='any', subset=['text', 'createdAt'])
print('The data have been reduced from {} tweets to {} tweets.'.format(len(twh), len(tw1)))

In [ ]:
pr('Removing bad dates...')
twhCleanDate = tw1[tw1['createdAt'].str.len() == 19]
pr('Finished.')

In [ ]:
pr('Starting to examine dates...')
import warnings
warnings.filterwarnings('ignore')
datetime_serie = twhCleanDate['createdAt'].convert_objects(convert_dates='coerce')
dateNotConvertible = datetime_serie[pd.isnull(datetime_serie)]
warnings.filterwarnings('default')
pr('There are {} dates that cannot be transformed.'.format(len(dateNotConvertible)))

In [ ]:
pr('Starting copy...') # (to avoid transformation problems)
tw5 = twhCleanDate.copy()
pr('Converting to datetime...')
tw5['createdAt'] = pd.to_datetime(twhCleanDate['createdAt'])
pr('Setting up new indices...')
tw5.index = tw5['createdAt']
pr('Deleting old "createdAt" column...')
del tw5['createdAt']
pr('Done!')
tw5.head(2)

In [ ]:
tw5['hashtag'][:6]

## Let's put one hashtag per row

We will make a dataframe with one row = one hashtag. This will be done by going through the dataframe, and making in parallel a list of rows (with 1 hashtag per row) that needs to be added to the old dataframe.

In [ ]:
addedHashtagsRowsList = []
def multiplyHashtagRows(row, columns):
    '''
    Examine each row. If there are multiple hashtags, it will return the first one.
    (so the first one will replace the list of hashtags in the df). Then for all the next ones,
    it will make a copy of the row in the addedHashtagsRowsList, (in a dictionary format).
    So this dictionary can in the end be transformed in a DF and added to the original DF.
    (The speed is increased a lot by doing it this way!)
    '''
    htList = row.hashtag
    if len(htList) > 1:
        ## Making the dictionary
        addedHashtag = {}
        addedHashtag['createdAt'] = row.name #the df index
        for col in columns:
            addedHashtag[col] = row[col]
        ## Copying the dict for each hashtag
        i = 1
        while i < len(htList) :
            deepCopy = copy.deepcopy(addedHashtag)
            deepCopy['hashtag'] = htList[i]
            addedHashtagsRowsList.append(deepCopy)
            i+=1
    return htList[0] # return the first hashtag

In [ ]:
addedHashtagsRowsList = []
tw5_1 = tw5.copy()
pr('Multiplying the hashtag rows... (around 10 min)')
tw5_1['hashtag'] = tw5.apply(multiplyHashtagRows, args=[tw5.columns,], axis=1)
pr('Finished! {} rows will be added to the dataframe!'.format(len(addedHashtagsRowsList)))

In [ ]:
pr('Starting to make the new dataframe with additionnal rows..')
addedHashtagsDf = pd.DataFrame(addedHashtagsRowsList)
addedHashtagsDf.set_index(['createdAt'], inplace=True)
pr('Starting to append the two df... Old df size = {}'.format(len(tw5_1)))
tw6 = tw5_1.append(addedHashtagsDf)
pr('Done! New df size = {}'.format(len(tw6)))

In [ ]:
print('Example hahshtag:')
tw6[tw6['hashtag'] == addedHashtagsRowsList[0]['hashtag']].head(3)

In [ ]:
tw6.hashtag.head()

## Grouping per hashtags per day

In order to provide visualisation to the other team, we needed to give ocation information to all tweets. So we did not process tweets without longitude or latitude.

In [ ]:
tw6.dropna(subset=['longitude'], inplace=True)
tw6.dropna(subset=['latitude'], inplace=True)

In [ ]:
tw6.latitude = tw6.latitude.apply(float)
tw6.longitude = tw6.longitude.apply(float)

Merging information of all tweet containing a particular hastag turned out to be really useful to detect an event. For column of String type, we used a join with a delimiter as balow.

In [ ]:
delimiter = '_$$$_'
str_join = lambda x: delimiter.join(x)

Function that applies to a dataframe will group each row by day and aggregate all its content:

In [ ]:
def aggDate(df):
    temp = df.groupby(df.index.map(lambda x: x.date))
    groupedDf = temp.agg({  'text' : str_join,
                            'longitude' : np.median,
                            'latitude' : np.median,
                            'hashtag' : lambda x: x.iloc[0], ## the first occurence
                            'numberOfTweets' : 'count',
                            'userId' : pd.Series.nunique})
    # rename userId column to a more representative name
    return groupedDf

In [ ]:
# Creation of a dictionnary {'hashtag' : 'dataframe containing all tweet with the hashtag'}
print('Dictionary with hashtags dataframes:')
dictionary[list(dictionary.keys())[5]].head(4)

## Event detection (with elimination of recurrent events)

Parameters that define events:

In [ ]:
## Parameters of an event:
MIN_TOT_NB_TWEETS = 20 ## The hashtag must have happened at least this number of times to be considered.
MIN_NB_DAYS_WITH_HASHTAGS = 3 ## The hashtags must appear at least this number of different days to be considered.
MIN_NB_TWEETS_DURING_EVENT = 7 ## To be considered an event, the hashtag must happen at least this nb of times during the day.
THRESHOLD_ANOMALY_FACTOR = 2.5 ## The occurence of a hashtag during a single day must be above the mean by this FACTOR
                             ## multiplied by the std to be considered as an anomaly.
MAX_DURATION_OF_EVENT = timedelta(days=30) ## The maximum number of days we consider an event can happen
MIN_DURATION_BEFORE_NEW_EVENT = timedelta(days=304) ## (= 10 months) The min time that should pass before an event can happen
                                                    ## again and still be considered as event (ie. Christmas is an event
                                                    ## each year)
MIN_NUMBER_DIFFERENT_USER = 2 # To state that an event occured, a minimum number of different users should have tweeted about it

Helper functions to detect recurrent events that should be removed:

In [ ]:
def isSpecificEventListIllegal(detectedEventDateList):
    '''
    Return true if the list of dates contain illegal tupples of events, so if the event is recurrent
    which would mean it is not a real event.
    '''
    def datesAreIllegal(date1, date2, date3):
        '''
        Return true if the 3 dates are not to be considered as regular events.
        '''
        ## Return if the difference is too small to be considered as 2 different events
        def diffIsSmall(timeDiff):  
            return timeDiff < MAX_DURATION_OF_EVENT

        ## Return true if the difference is not big enough to be an annual event.
        def isDiffSuspect(timeDiff):
            return timeDiff < MIN_DURATION_BEFORE_NEW_EVENT   

        diff1 = abs(date1 - date2)
        diff2 = abs(date2 - date3)
        diff3 = abs(date3 - date1)

        ## The difference is too small, it must be the same event
        if diffIsSmall(diff1) or diffIsSmall(diff2) or diffIsSmall(diff3):
            return False

        ## If there are at least 2 out of 3 suspect difference, then the dates are illegal
        if isDiffSuspect(diff1):
            return isDiffSuspect(diff2) or isDiffSuspect(diff3)
        else:
            return isDiffSuspect(diff2) and isDiffSuspect(diff3)
    
    ## MAIN FUNCTION : ##
    # Go through the list of events and try all "triples" to see if there is any illegal triples. This is a quickly done
    # code to do that. Code complexity bellow is in O(k^3), with k being the size of the list. We will apply this function
    # to n list so we will have an overall complexity in O(n*k^3). We can consider however that each list will
    # be small so k can be considered as constant and therefore the overall complexity will be in O(n).
    for i in range(len(detectedEventDateList) - 2):
        for j in range(i, len(detectedEventDateList) - 1):
            for k in range(j, len(detectedEventDateList)):
                if datesAreIllegal(detectedEventDateList[i], detectedEventDateList[j], detectedEventDateList[k]):
                    return True
    return False

Main method to detect event according to all our criterias.

In [ ]:
pr('Starting to compute {} dict items to detect event. (4 min)'.format(len(dictionary)))
nbOfEventDetected = 0
count = 0
printingValue = int(len(dictionary) / 10)
for [h,df] in dictionary.items():
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/len(dictionary)*100))
    df['event'] = False
    if len(df) > MIN_NB_DAYS_WITH_HASHTAGS:
        if df['numberOfTweets'].sum() > MIN_TOT_NB_TWEETS:
            if df['userId'][0] >= MIN_NUMBER_DIFFERENT_USER:
                threshold = df['numberOfTweets'].mean() + THRESHOLD_ANOMALY_FACTOR * df['numberOfTweets'].std()
                df['event'] = df.numberOfTweets.apply(lambda x: x > threshold and x > MIN_NB_TWEETS_DURING_EVENT)
            
            ## Remove recurrent events:
            detectedEventDf = df[df['event']]
            if len(detectedEventDf) > 2 and isSpecificEventListIllegal(detectedEventDf.index):
                df['event'] = False
            nbOfEventDetected += len(df[df['event']])
pr('Finished! Number of events detected = {}'.format(nbOfEventDetected))

In [ ]:
# pr('Starting to compute {} dict items to detect event. (4 min)'.format(len(dictionary)))
# nbOfEventDetected = 0
# count = 0
# printingValue = int(len(dictionary) / 10)
# for [h,df] in dictionary.items():
#     count += 1
#     if count % printingValue == 0:
#         pr("{:.0f}%".format(count/len(dictionary)*100))
#     df['event'] = False
#     if len(df) > MIN_NB_DAYS_WITH_HASHTAGS:
#         if df['numberOfTweets'].sum() > MIN_TOT_NB_TWEETS:
#             threshold = df['numberOfTweets'].mean() + THRESHOLD_ANOMALY_FACTOR * df['numberOfTweets'].std()
#             df['event'] = df.numberOfTweets.apply(lambda x: x >= threshold and x >= MIN_NB_TWEETS_DURING_EVENT)
            
#             ## Remove recurrent events:
#             detectedEventDf = df[df['event']]
#             if len(detectedEventDf) > 2 and isSpecificEventListIllegal(detectedEventDf.index):
#                 df['event'] = False
#             nbOfEventDetected += len(df[df['event']])
# pr('Finished! Number of events detected = {}'.format(nbOfEventDetected))

## Merging close events and grouping into single event dataframe

Here, we have a function that is going to merge events that are considered as too "close" to each other to be considered individually.

In [ ]:
def mergeCloseEvents(rowsList):
    '''
    Take a list of dictionary, where each dictionary is a "row" of the event df, which contained detected events.
    It will process the list to detect event that are close and merge them together.
    Return : the processed list of event.
    '''
    
    def areCloseEvents(event1, event2):
        '''
        Return true is 2 events dates are defined as "close"
        '''
        return abs(event1['date'] - event2['date']) < MAX_DURATION_OF_EVENT
        
    def mergeCloseEventsSublist(closeEventList):
        '''
        This will be applied to each close event sublist. It will merge all events into one unique event.
        The event will consist of the total number of tweets, with the concatenation of the tweet texts and the mean
        of longitude/latitude. A meanDate will be defined as a ponderated mean between all dates.
        The final date will be the one that is in the closeEventList and is closest to this mean date.
        We did this to keep the meaning of the date if it had some, and not have some meaningless "mean-date".
        '''
        latitude = 0
        longitude = 0
        numberOfTweets = 0
        text = ""
        originalDate = closeEventList[0]['date']
        dateDiff = timedelta(days=0)
        first = True        
        for tweet in closeEventList:
            longitude += tweet['longitude']
            latitude += tweet['latitude'] 
            numberOfTweets += tweet['numberOfTweets']
            if first:
                text = tweet['text']
                first = False
            else:
                text += delimiter + tweet['text']
                dateDiff = dateDiff + (tweet['date'] - originalDate) * tweet['numberOfTweets']

        ## It is multiplied by 2 then soustracted to round correctly to the nearest day
        meanDate = originalDate + 2* dateDiff / numberOfTweets - dateDiff / numberOfTweets        
        latitude = latitude / len(closeEventList)
        longitude = longitude / len(closeEventList)
        
        ## We are going to detect the event the closest to the mean date
        minSelectedDate = closeEventList[0]['date']
        minDistance = abs(closeEventList[0]['date'] - meanDate)
        for tweet in closeEventList:
            if abs(tweet['date'] - meanDate) < minDistance:
                minSelectedDate = tweet['date']    
        
        return {'date': minSelectedDate, 'hashtag': closeEventList[0]['hashtag'], 'text': text,
                    'longitude': longitude, 'latitude':latitude, 'numberOfTweets': numberOfTweets, }
    
    ############ -----  MAIN METHOD  ----- ############
    
    ## If the list is big enough, go through the list and form an export list and merge elements that needs to.
    if len(rowsList) < 2:
        return rowsList
    else:
        firstLastPosOfItemsToMerge = []
        sortedRowsList = sorted(rowsList, key=itemgetter('date')) 
        exportedEventList = []
        ## This goes through the *sorted* list and add the pair of indices (first indice and last indice) where events 
        ## that should be merged appear.
        lastEventWasClose = False
        firstItem = -1
        for i in range(0, len(sortedRowsList)-1):
            if areCloseEvents(sortedRowsList[i], sortedRowsList[i+1]):
                if not lastEventWasClose: # So it is the first pairs of the sublist of close events in the whole list
                    firstItem = i
                    lastEventWasClose = True
            else:
                if lastEventWasClose: # So the list has just ended.
                    exportedEventList.append(mergeCloseEventsSublist(sortedRowsList[firstItem:i+1]))
                    lastEventWasClose = False
                else: # The element is by itself, let's append it
                    exportedEventList.append(sortedRowsList[i])  
        if lastEventWasClose: # If there were events to merge till the last elem of list
            exportedEventList.append(mergeCloseEventsSublist(sortedRowsList[firstItem:len(sortedRowsList)]))
        else:
            exportedEventList.append(sortedRowsList[len(sortedRowsList)-1])
    
    return exportedEventList

This function will be applied to each dataframe. If a row is detected as an event, it will be added to the locaRowsList which will be used to make a general dataframe of all the events.

In [ ]:
localRowsList = []
def applyToMakeEventDf(row):
    if row.event:
        rowToAdd = {'date': row.name, 'hashtag': row.hashtag, 'text': row.text,
                    'longitude': row.longitude, 'latitude':row.latitude, 'numberOfTweets': row.numberOfTweets, }
        global localRowsList
        localRowsList.append(rowToAdd)

In [ ]:
eventRowsList = []
localRowsList = []
count = 0
printingValue = int(len(dictionary) / 10)

pr('Starting to make event df with {} dataframes. (around 6 min)'.format(len(dictionary)))
for h, df in dictionary.items():
    global localRowsList
    localRowsList = []
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/len(dictionary)*100))
        
    df.apply(applyToMakeEventDf, axis=1)
    mergedList = mergeCloseEvents(localRowsList) # merging close events
    eventRowsList += mergedList

pr('Making new dataframe.')
new_events = pd.DataFrame(eventRowsList)
new_events.set_index(['date'], inplace=True)
pr('Finished! Dataframe with {} rows'.format(len(new_events)))

In [ ]:
print('Events dataframe:')
new_events.head(50)

In [ ]:
print('Linked dataframe of all days:')
dictionary[new_events.iloc[0].hashtag].head(10)

## Exporting data

As we worked with another team, we needed a way to communicate them our detection. We used a JSON with all the information.

In [ ]:
total_number_of_events = len(new_events)
print('There are {} events.'.format(total_number_of_events))

In [ ]:
e_df = new_events.copy()
e_df['date'] = e_df.index
e_df.index = [i for i in range (len(e_df))]
e_df.head(1)

We are going to generate the right datetimes for the jsons:

In [ ]:
epoch_dt = datetime(1970, 1, 1)
def to_utc(date):
    d_dt = datetime.combine(date, datetime.min.time())
    return int((d_dt - epoch_dt).total_seconds()*1000)

In [ ]:
def convert_to_unix_time(record):
    datetime_index = pd.DatetimeIndex([datetime(record['year'], record['month'], 1)])
    unix_time_index = datetime_index.astype(np.int64) // 10**6
    return unix_time_index[0]

In [ ]:
pr('Converting dates...')
e_df['year'] = e_df['date'].apply(lambda x: x.year)
e_df['month'] = e_df['date'].apply(lambda x: x.month)
e_df['utc_date'] = e_df['date'].apply(lambda x: to_utc(x))
e_df['unix_time'] = e_df.apply(convert_to_unix_time, axis=1)
pr('Done.')
e_df.head(1)

The generation of a JSON is easier from a dictionary than from a dataframe. Also, the other team we worked with asked us to group events by months.

In [ ]:
# Grouping by months
e_gb_month = e_df.groupby(e_df.unix_time)

In [ ]:
# Generation of the dictionary for the final JSON
pr('Making event list...')
months = []
for month, df in e_gb_month:
    days = []
    for i in range (len(df)):
        ht = df.iloc[i]['hashtag']
        lat = df.iloc[i]['latitude']
        lon = df.iloc[i]['longitude']
        t_num = df.iloc[i]['numberOfTweets']
        tweets = df.iloc[i]['text'].split(delimiter)
        date = df.iloc[i]['utc_date']
        
        data_unit = { 'name': ht
                    , 'latitude' : lat
                    , 'longitude' : lon
                    , 'tweets' : tweets
                    , 'number_of_tweets' : str(t_num)
                    , 'date' : int(date)}
        days.append(data_unit)
    
    curr_month = {'date': int(month), 'data' : days}
    months.append(curr_month)

final_events = {'events' : months}
pr('Done.')

Creation of the final JSON

In [ ]:
exportFilename = 'export_twitter_events_' + datetime.now().strftime("%Y-%m-%d_%Hh%Mmin%S") + \
'_' + str(total_number_of_events)+ '_events.json'
exportPath =  os.path.join('data', exportFilename)

pr('Exporting to json...')
with open(exportPath, 'w') as f:
     json.dump(final_events, f)
pr('Export done. File "{}" has been created.'.format(exportFilename))